# Lid Driven Cavity Background
### Using NVIDIA Syms we will solve the LDC problem using PINNs
#### Steps are like:
1. Generate a 2D geometry using Modulus SYM geometry Module
2. Setup the boundary conditions
3. Select the flow equations to be solved
4. Interpret diffent losses and tune the network after followed by a basic preprocessing

The problem involves a 0.1 m square cavity with stationary walls on three sides and a top wall moving right at 1 m/s. The Reynolds number, indicating flow type, is calculated based on flow speed and cavity height. For this setup, the Reynolds number is 10, indicating laminar flow.


The **governing equations** for fluid dynamics in this context are the **Navier-Stokes equations**, which describe the motion of viscous fluid substances. In two dimensions, the Navier-Stokes equations can be expressed as follows:

##### 1. **Continuity Equation**
The continuity equation ensures mass conservation:
$$
\frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \mathbf{u}) = 0
$$
Where:
- $\rho$ is the fluid density.
- $\mathbf{u} = (u, v)$ is the velocity vector, with $u$ and $v$ being the velocity components in the $x$ and $y$ directions, respectively.


##### 2. **Momentum Equations**
The momentum equations describe the change in momentum due to various forces:
$$
\frac{\partial (\rho \mathbf{u})}{\partial t} + \nabla \cdot (\rho \mathbf{u} \otimes \mathbf{u}) = -\nabla p + \mu \nabla^2 \mathbf{u} + \mathbf{f}
$$
Where:
- $p$ is the pressure.
- $\mu$ is the dynamic viscosity of the fluid.
- $\nabla^2$ is the Laplacian operator, representing viscous diffusion.
- $\mathbf{f}$ represents body forces (e.g., gravity).

##### Simplified Form
In many cases, particularly for incompressible flows (where the density is constant), the equations simplify to:
1. **Continuity** (for incompressible flow):
$ \nabla \cdot \mathbf{u} = 0 $

2. **Momentum**:
$$
\frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} = -\frac{1}{\rho} \nabla p + \nu \nabla^2 \mathbf{u}
$$
Where:
- $ \nu = \frac{\mu}{\rho} $ is the kinematic viscosity.


The Navier-Stokes equations encapsulate the fundamental principles of fluid motion, including conservation of mass and momentum.

In [1]:

! pip install nvidia-modulus.sym --no-deps
! pip install "hydra-core>=1.2.0" "termcolor>=2.1.1" "chaospy>=4.3.7" "Cython==0.29.28" "numpy-stl==2.16.3" "scikit-learn==1.0.2" "symengine>=0.10.0" "sympy==1.12" "timm>=1.0.3" "torch-optimizer==0.3.0" "transforms3d==0.3.1" "typing==3.7.4.3" "pillow==10.0.1" "notebook==6.4.12" "mistune==2.0.3" "pint==0.19.2" "tensorboard>=2.8.0"

  Using cached nvidia_modulus.sym-1.6.0-py3-none-any.whl (291 kB)



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached hydra_core-1.3.2-py3-none-any.whl (154 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached chaospy-4.3.16-py3-none-any.whl (254 kB)
  Using cached Cython-0.29.28-py2.py3-none-any.whl (983 kB)
  Using cached numpy-stl-2.16.3.tar.gz (772 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.2 MB 1.3 MB/s eta 0:00:06
      --------------------------------------- 0.1/7.2 MB 1.3 MB/s eta 0:00:06
     - -------------------------------------- 0.2/7.2 MB 1.9 MB/s eta 0:00:04
     -- ------------------------------------- 0.5/7.2 MB 2.8 MB/s eta 0:00:03
     ---- ----------------------------------- 0.8/7.2 MB 3.7 MB/s eta 0:00:02
     ---- ----------------------------------- 0.8/7.2 MB 3.7 MB/s eta 0:00:02
     ---- ----------------------------------- 0.8/7.2 MB 3.0 MB/s eta 0:00:03
  

  DEPRECATION: transforms3d is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: antlr4-python3-runtime is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: typing is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discuss

: 

In [1]:
# Import all the  libraries
import os
import warnings

from sympy import Symbol, Eq, Abs
import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.key import Key
from modulus.sym.eq.pdes.navier_stokes import NavierStokes
from modulus.sym.utils.io import (
    csv_to_dict,
    ValidatorPlotter,
    InferencerPlotter,
)

ModuleNotFoundError: No module named 'pint'